In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pdb

In [2]:
years = range(2016,2021)
months = ['04', '10']
home_page = 'https://www.churchofjesuschrist.org'

var_names = ['year', 'month', 'speaker', 'title', 'text']
conf_data = []

for year in years:
    for month in months:
        # Download conference html file for given date
        conf_page = requests.get(f'{home_page}/general-conference/{year}/{month}?lang=eng')
        conf_soup = BeautifulSoup(conf_page.content, 'html.parser')
        
        # Information for each talk in given conference
        links = conf_soup.find_all(class_="lumen-tile__link")
        for link_contents in links:
            try:
                #speaker_name = link_contents.img.get('alt')
                speaker_name = link_contents.find('div', class_='lumen-tile__content')
                subdir = link_contents.get('href')
                
                # Skip talk if there is no html site with talk text (usually general women's meeting)
                if re.search('media', subdir) != None:
                    print('No text available:', f'{home_page}{subdir}')
                    continue
            except:
                print('Exception: Problem getting speaker name')
                print(f'{month} {year} Conference')
                print(link_contents)
                pdb.set_trace()
            
            # Get talk text
            talk_page = requests.get(f'{home_page}{subdir}')
            talk_soup = BeautifulSoup(talk_page.content, 'html.parser')
            talk_title = talk_soup.h1.string
            
            # Replace lettered references with reference content
            references = talk_soup.find_all(id=re.compile('note[0-9]+'))
            
            # Combine references that are from same note
            n_refs = int( re.search('(?<=note)[0-9]+', references[-1].get('id')).group(0) )
            if len(references) > n_refs:
                pdb.set_trace()
                new_references = references.copy()
                i = 0
                while i < len(new_references)-1:
                    # combine parts from same note
                    ref1 = new_references[i]
                    ref2 = new_references[i+1]
                    ref1_note = re.search('note[0-9]+', ref1.get('id')).group(0)
                    ref2_note = re.search('note[0-9]+', ref2.get('id')).group(0)
                    if ref1_note == ref2_note:
                        string1 = ' '.join(ref1.stripped_strings)
                        string2 = ' '.join(ref2.stripped_strings)
                    
                    i += 1
            
            talk_refs = talk_soup.find_all(href=re.compile('note[0-9]+'))
            for i, ref in enumerate(references):
                new_string = ' '.join(ref.stripped_strings)
                '''if ref.find('a') != None:
                    new_string = ref.find('a').string
                else:
                    new_string = ' '.join(ref.stripped_strings)'''
                # Replace number in talk body text with formatted citation, be it scripture or other reference
                try:
                    talk_refs[i].string.replace_with(f' ({new_string}) ')
                except:
                    print('Exception: Problem replacing reference string')
                    print(f'{month} {year} Conference')
                    print(f'Speaker: {speaker_name}, Talk: {talk_title}')
                    print(f'Talk total references: {len(references)}, Currently on: {i}')
                    pdb.set_trace()
            '''
            # FOR DEBUGGING
            print(f'{speaker_name}, {talk_title}')
            for i, ref in enumerate(references):
                print(f'Reference {i}:', ref)
                print(f'Found in Text:', talk_refs[i], '\n')
            '''
            # Get talk body text after adding references
            try:
                talk_text = talk_soup.find(class_="body-block").get_text()
            except:
                print('Exception: Problem obtaining body text')
                print(f'{month} {year} Conference')
                print(f'Speaker: {speaker_name}, Talk: {talk_title}')
                pdb.set_trace()
            conf_data.append((year, month, speaker_name, talk_title, talk_text))
            
conf_df = pd.DataFrame(conf_data, columns=var_names)
conf_df.to_csv('conference.csv', index=False)

No text available: https://www.churchofjesuschrist.org/general-conference/2016/04/media/4810651550001?lang=eng
No text available: https://www.churchofjesuschrist.org/general-conference/2016/04/media/4810668496001?lang=eng
> <ipython-input-2-555d47f8c081>(44)<module>()
-> new_references = references.copy()
(Pdb) talk_title
'“I Was a Stranger”'
(Pdb) month
'04'
(Pdb) year
2016
(Pdb) exit


BdbQuit: 

In [4]:
for ref in references:
    print(ref)

<p data-aid="29472267" id="note1_p1">Emma Smith, in <em>Daughters in My Kingdom: The History and Work of Relief Society </em>(2011), 14.</p>
<p data-aid="29472268" id="note2_p1">Brigham Young, in <em>Daughters in My Kingdom, </em>36.</p>
<p data-aid="29472269" id="note3_p1"><em>Daughters in My Kingdom, </em>36–37.</p>
<p data-aid="29472270" id="note4_p1">Brigham Young, in James E. Faust, “<a class="" href="/#note4">Go Bring Them in from the Plains</a>,” <em>Ensign,</em> July 1997, 7; <em>Liahona,</em> Nov. 1997, 7; see also LeRoy R. and Ann W. Hafen, <em>Handcarts to Zion: The Story of a Unique Western Migration 1856–1860</em> (1960), 139.</p>
<p data-aid="29472271" id="note5_p1">Lucy Meserve Smith, in Jill Mulvay Derr and others, eds., <em>The First Fifty Years of Relief Society: Key Documents in Latter-day Saint Women’s History </em>(2016), 217, 218, spelling and punctuation standardized; see also <em>Daughters in My Kingdom, </em>37.</p>
<p data-aid="29472272" id="note6_p1">See “Fac